# Setups/Imports

In [1]:
import os
import numpy as np
import scipy as sp
import scipy.sparse as sprse
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import copy

/opt/tljh/user/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
matplotlib.use('Agg') # to force use .show()

## Config

Global parameters for the file.

In [3]:
class config:
    fixed_seq_len = 216
    path = '/data/users/jupyter-dam724/datadump(colliding)'
    images = '/data/users/jupyter-dam724/imagedump(colliding)'
    solver = 'ros2'
    vec_size = 40000
    seperate_modes = False
    remove_density = True

# Dataset

Cache file paths and sequence lengths.

In [4]:
def get_num_instances(folder):
    inst = set()
    
    for filename in os.listdir(folder):
        if filename.split('_')[2] in inst:
            continue
        inst.add(int(filename.split('_')[2]))
    
    return inst

In [5]:
def get_instance_lens(folder, instances):
    inst_len_cache = {}
    
    for instance in instances:
        A = os.path.join(folder, f'{config.solver}_A_{instance}_')
        b = os.path.join(folder, f'{config.solver}_b_{instance}_')
        x = os.path.join(folder, f'{config.solver}_x_{instance}_')
        
        for seq in range(0, config.fixed_seq_len+1):
            A_seq = A + f'{seq}.npz'
            b_seq = b + f'{seq}.npy'
            x_seq = x + f'{seq}.npy'
            
            if not (os.path.isfile(A_seq) and os.path.isfile(b_seq) and os.path.isfile(x_seq)):
                # If finished early use timesteps 10 before.
                if seq != config.fixed_seq_len:
                    inst_len_cache[instance] = seq-10
                else:
                    inst_len_cache[instance] = seq-1
                break
                
    return inst_len_cache

In [6]:
def get_instance_paths(folder, instances, inst_len_cache):
    file_cache = {}
        
    for instance in instances:
        for seq in range(inst_len_cache[1]+1):
            A_file = f'A_{instance}_{seq}'
            b_file = f'b_{instance}_{seq}'
            x_file = f'x_{instance}_{seq}'
            
            file_cache[A_file] = os.path.join(folder, f'{config.solver}_{A_file}.npz')
            file_cache[b_file] = os.path.join(folder, f'{config.solver}_{b_file}.npy')
            file_cache[x_file] = os.path.join(folder, f'{config.solver}_{x_file}.npy')
    
    return file_cache

In [7]:
def get_image_paths(folder, instances, inst_len_cache):
    file_cache = {}
        
    for instance in instances:
        for seq in range(inst_len_cache[1]+1):
            image_file = f'{instance}_bubble_3_{seq:08d}.png'
            file_cache[f'image_{instance}_{seq}'] = os.path.join(folder, image_file)
    
    return file_cache

In [8]:
def get_full_instance(folder, file_cache, inst_len_cache, instance):
    full_seq = np.zeros((config.vec_size, inst_len_cache[instance]))
    
    for seq in range(inst_len_cache[instance]):
        file = f'x_{instance}_{seq}'
            
        full_seq[:, seq] = np.load(file_cache[file])
        
    return full_seq

In [9]:
instances = get_num_instances(config.path)
seq_len_cache = get_instance_lens(config.path, instances)
file_cache = get_instance_paths(config.path, instances, seq_len_cache)
image_cache = get_image_paths(config.images, instances, seq_len_cache)

In [10]:
seq_len_cache

{1: 215,
 2: 215,
 3: 215,
 4: 215,
 5: 215,
 6: 215,
 7: 215,
 8: 215,
 9: 215,
 10: 192,
 11: 215,
 12: 215,
 13: 215,
 14: 215,
 15: 215,
 16: 215,
 17: 215,
 18: 215,
 19: 215,
 20: 161,
 21: 215,
 22: 215,
 23: 215,
 24: 215,
 25: 215,
 26: 215,
 27: 215,
 28: 215,
 29: 156,
 30: 215,
 31: 215,
 32: 215,
 33: 215,
 34: 215,
 35: 215,
 36: 215,
 37: 215,
 38: 215,
 39: 215,
 40: 182,
 41: 215,
 42: 215,
 43: 215,
 44: 215,
 45: 215,
 46: 189,
 47: 215,
 48: 215,
 49: 174,
 50: 215,
 51: 215,
 52: 215,
 53: 199,
 54: 177,
 55: 215,
 56: 215,
 57: 215,
 58: 215,
 59: 215,
 60: 215,
 61: 215,
 62: 215,
 63: 215,
 64: 215,
 65: 174,
 66: 215,
 67: 215,
 68: 201,
 69: 215,
 70: 215,
 71: 215,
 72: 215,
 73: 215,
 74: 215,
 75: 215,
 76: 215,
 77: 215,
 78: 215,
 79: 215,
 80: 215,
 81: 186,
 82: 215,
 83: 215,
 84: 215,
 85: 187,
 86: 215,
 87: 215,
 88: 215,
 89: 215,
 90: 215,
 91: 215,
 92: 215,
 93: 215,
 94: 215,
 95: 215,
 96: 215,
 97: 215,
 98: 215,
 99: 196,
 100: 215,
 101: 21

## Sanity Check 01

Ensure the dataset is setup correctly. *View the changing systems.*

In [11]:
def image_animation(instance, seq_len_cache, image_cache):
    fig, ax = plt.subplots()
    ax.axis('off')
    frame_display = ax.imshow(np.array(Image.open(image_cache[f'image_{instance}_{0}'])), interpolation='antialiased')

    def update_frame(frame_number):
        next_image = np.array(Image.open(image_cache[f'image_{instance}_{frame_number}']))
        frame_display.set_array(next_image)
        return [frame_display]

    ani = animation.FuncAnimation(fig, update_frame, frames=seq_len_cache[instance], interval=1, blit=True)
    
    print(f'Now saving animation for instance ({instance}) with ({seq_len_cache[instance]}) sequences.')

    output_path = f'animations/instance_{instance}.gif'
    ani.save(output_path, writer='Pillow', fps=20)

*Error check 01-30, instances that failed on gpu(0).*

[11, 15]

[123, 124, 133, 134, 143, 144, 153, 154, 163, 164, 173, 174, 183, 184, 193, 194, 203, 204, 213, 214, 223, 224, 233, 234]

In [12]:
image_animation(15, seq_len_cache, image_cache)

MovieWriter Pillow unavailable; using Pillow instead.


Now saving animation for instance (15) with (215) sequences.


In [13]:
def get_indpendent_vars(x_vec):
    x_temp = copy.deepcopy(x_vec)
    
    x_density = x_temp[:10000]
    x_temp[10000:20000] = x_temp[10000:20000] / x_density
    x_temp[20000:30000] = x_temp[20000:30000] / x_density
    x_temp[30000:] = x_temp[30000:] / x_density
    
    return x_temp

In [14]:
def get_dependent_vars(x_vec):
    x_temp = copy.deepcopy(x_vec)
    
    x_density = x_temp[:10000]
    x_temp[10000:20000] = x_temp[10000:20000] * x_density
    x_temp[20000:30000] = x_temp[20000:30000] * x_density
    x_temp[30000:] = x_temp[30000:] * x_density
    
    return x_temp

In [15]:
def plot_timestep(x_vec, instance):
    X = x_vec.reshape(4, 100, 100)
    
    density = X[0]
    h_vcity = X[1] 
    v_vcity = X[2] 
    temp = X[3] 
        
    num_levels = 25
    
    def compute_graphics(mat, levels, typ, instance):
        plt.clf()
                
        contour_levels = np.linspace(mat.min(), mat.max(), num_levels)
        colors = plt.cm.jet(np.linspace(0, 1, num_levels))  
        plt.contourf(mat, levels=contour_levels, colors=colors, linestyles='-')
        plt.colorbar()

        plt.savefig(f'plots/test_{instance}_{typ}.png')  
        
        plt.show()
    
    compute_graphics(density, num_levels, 'density', instance)
    compute_graphics(h_vcity, num_levels, 'horizontalVelocity', instance)
    compute_graphics(v_vcity, num_levels, 'verticalVelocity', instance)
    compute_graphics(temp, num_levels, 'temperature', instance)

In [16]:
x_vec = np.load(file_cache['x_1_10'])
plot_timestep(get_indpendent_vars(x_vec), 1)

# Dynamic Mode Decomposition

\begin{align}
...
\end{align}

Look at the modes of instances and try to make inferences based on eigenvalues.

In [17]:
def full_mode_animation(Phi, var, ny, nx, instance, modes):
    X = Phi.reshape(var, ny, nx, modes)
    
    density = X[0]
    h_vcity = X[1] 
    v_vcity = X[2] 
    temp = X[3] 
        
    num_levels = 25
    
    def compute_animation(mat, levels):
        fig, ax = plt.subplots()
        ax.axis('off')
        
        contour_levels = np.linspace(mat[:, :, 0].min(), mat[:, :, 0].max(), num_levels)
        colors = plt.cm.jet(np.linspace(0, 1, num_levels))  
        ax.contourf(mat[:, :, 0], levels=contour_levels, colors=colors, linestyles='-')

        def update_frame(mode):
            ax.clear()

            contour_levels = np.linspace(mat[:, :, mode].min(), mat[:, :, mode].max(), num_levels)
            colors = plt.cm.jet(np.linspace(0, 1, num_levels))  
            contours = ax.contourf(mat[:, :, mode], levels=contour_levels, colors=colors, linestyles='-')
            return contours.collections

        return animation.FuncAnimation(fig, update_frame, frames=mat.shape[2], interval=1, blit=True)
    
    print(f'Now saving animations out Density/Horizontal Velocity/Vertical Velocity/Temperature for instance ({instance}) with ({Phi.shape[1]}) modes.')
    
    ani = compute_animation(density, num_levels)
    output_path = f'animations/instance_{instance}_full_modes_density_{Phi.shape[1]}.gif'
    ani.save(output_path, writer='Pillow', fps=1)
    
    ani = compute_animation(h_vcity, num_levels)
    output_path = f'animations/instance_{instance}_full_modes_horizontalVelocity_{Phi.shape[1]}.gif'
    ani.save(output_path, writer='Pillow', fps=1)
    
    ani = compute_animation(v_vcity, num_levels)
    output_path = f'animations/instance_{instance}_full_modes_verticalVelocity_{Phi.shape[1]}.gif'
    ani.save(output_path, writer='Pillow', fps=1)
    
    ani = compute_animation(temp, num_levels)
    output_path = f'animations/instance_{instance}_full_modes_temperature_{Phi.shape[1]}.gif'
    ani.save(output_path, writer='Pillow', fps=1)

In [18]:
def partial_mode_animation(Phi, var, ny, nx, instance, modes, typ):
    X = Phi.reshape(var, ny, nx, modes)
        
    num_levels = 25
    
    def compute_animation(mat, levels):
        fig, ax = plt.subplots()
        ax.axis('off')
        
        contour_levels = np.linspace(mat[:, :, 0].min(), mat[:, :, 0].max(), num_levels)
        colors = plt.cm.jet(np.linspace(0, 1, num_levels))  
        ax.contourf(mat[:, :, 0], levels=contour_levels, colors=colors, linestyles='-')

        def update_frame(mode):
            ax.clear()

            contour_levels = np.linspace(mat[:, :, mode].min(), mat[:, :, mode].max(), num_levels)
            colors = plt.cm.jet(np.linspace(0, 1, num_levels))  
            contours = ax.contourf(mat[:, :, mode], levels=contour_levels, colors=colors, linestyles='-')
            return contours.collections

        return animation.FuncAnimation(fig, update_frame, frames=mat.shape[2], interval=1, blit=True)
    
    print(f'Now saving animations out {typ} for instance ({instance}) with ({Phi.shape[1]}) modes.')
    
    ani = compute_animation(X[0], num_levels)
    output_path = f'animations/instance_{instance}_partial_modes_{typ}_{Phi.shape[1]}.gif'
    ani.save(output_path, writer='Pillow', fps=1)

In [19]:
def svd(x):
    U, S, Vh = np.linalg.svd(x, full_matrices=False)
    V = Vh.conj().T
    S = np.diag(S)

    plt.figure()
    plt.semilogy(np.diag(S))
    plt.xlabel('Singular Value Index')
    plt.ylabel('Singular Values (log scale)')
    plt.savefig(f'plots/prev_svd.png')
    
    plt.close()
    
    return U, S, V

In [20]:
def dmd(x, U, S, V, x_prime, r, sort=True):
    U_r = U[:, :r]
    S_r = S[:r, :r]
    V_r = V[:, :r]
    
    A_tilde = U_r.T @ x_prime @ V_r @ np.linalg.inv(S_r)
    
    Lambda, W = np.linalg.eig(A_tilde)
    
    Phi = x_prime @ V_r @ np.linalg.inv(S_r) @ W
    
    if sort:
        sort_indices = np.argsort(np.abs(Lambda))[::-1]
        Lambda = Lambda[sort_indices]
        Phi = Phi[:, sort_indices]
    
    return Phi, Lambda

In [21]:
def dmd_analysis(file_cache, seq_len_cache, instance, mode_nums=215, start=0, end=0, animation=True, seperate_modes=False, remove_density=True):
    X = get_full_instance(config.path, file_cache, seq_len_cache, instance)
    
    if remove_density:
        X_density = X[:10000, :]
        X[10000:20000, :] = X[10000:20000, :] / X_density
        X[20000:30000, :] = X[20000:30000, :] / X_density
        X[30000:, :] = X[30000:, :] / X_density
        
    x = X[:, start:seq_len_cache[instance]-end-1]
    x_prime = X[:, start+1:seq_len_cache[instance]-end]
    
    r = min(mode_nums+1, x.shape[1])
    
    if seperate_modes:
        U, S, V = svd(x[:10000])
        Phi, Lambda = dmd(x[:10000], U, S, V, x_prime[:10000], r)
        partial_mode_animation(np.real(Phi), 1, 100, 100, instance, Phi.shape[1], 'density')
        
        U, S, V = svd(x[10000:20000])
        Phi, Lambda = dmd(x[10000:20000], U, S, V, x_prime[10000:20000], r)
        partial_mode_animation(np.real(Phi), 1, 100, 100, instance, Phi.shape[1], 'horizontal')
        
        U, S, V = svd(x[20000:30000])
        Phi, Lambda = dmd(x[20000:30000], U, S, V, x_prime[20000:30000], r)
        partial_mode_animation(np.real(Phi), 1, 100, 100, instance, Phi.shape[1], 'vertical')
        
        U, S, V = svd(x[30000:])
        Phi, Lambda = dmd(x[30000:], U, S, V, x_prime[30000:], r)
        partial_mode_animation(np.real(Phi), 1, 100, 100, instance, Phi.shape[1], 'temperature')
    else:
        U, S, V = svd(x)
        Phi, Lambda = dmd(x, U, S, V, x_prime, r)
    
        full_mode_animation(np.real(Phi), 4, 100, 100, instance, Phi.shape[1])

In [22]:
config.seperate_modes, config.remove_density = False, True

In [23]:
dmd_analysis(file_cache, seq_len_cache, 1, mode_nums=15, seperate_modes=config.seperate_modes, remove_density=config.remove_density)

MovieWriter Pillow unavailable; using Pillow instead.


Now saving animations out Density/Horizontal Velocity/Vertical Velocity/Temperature for instance (1) with (16) modes.


MovieWriter Pillow unavailable; using Pillow instead.
MovieWriter Pillow unavailable; using Pillow instead.
MovieWriter Pillow unavailable; using Pillow instead.


## DMD Prediction

Discrete-time prediction of the next time step.

In [24]:
def dmd_prediction(file_cache, seq_len_cache, instance, num_modes, start=0, end=0, seperate_modes=False, remove_density=True):    
    X = get_full_instance(config.path, file_cache, seq_len_cache, instance)
    
    if remove_density:
        print('Removing desnity dependence.')
        X_density = X[:10000, :]
        X[10000:20000, :] = X[10000:20000, :] / X_density
        X[20000:30000, :] = X[20000:30000, :] / X_density
        X[30000:, :] = X[30000:, :] / X_density
        
    x = X[:, start:seq_len_cache[instance]-end-1]
    x_prime = X[:, start+1:seq_len_cache[instance]-end]
    
    r = min(num_modes+1, x.shape[1])
    
    if seperate_modes:
        print('Seperating variables.')
        U, S, V = svd(x[:10000])
        Phi, Lambda = dmd(x[:10000], U, S, V, x_prime[:10000], r)
        x_new_density = X[:10000, seq_len_cache[instance]-end]
        x_pred_density = Phi @ np.diag(Lambda) @ np.linalg.pinv(Phi) @ x_new_density
        
        U, S, V = svd(x[10000:20000])
        Phi, Lambda = dmd(x[10000:20000], U, S, V, x_prime[10000:20000], r)
        x_new_horizonatal = X[10000:20000, seq_len_cache[instance]-end]
        x_pred_horizonatal = Phi @ np.diag(Lambda) @ np.linalg.pinv(Phi) @ x_new_horizonatal
        
        U, S, V = svd(x[20000:30000])
        Phi, Lambda = dmd(x[20000:30000], U, S, V, x_prime[20000:30000], r)
        x_new_vertical = X[20000:30000, seq_len_cache[instance]-end]
        x_pred_vertical = Phi @ np.diag(Lambda) @ np.linalg.pinv(Phi) @ x_new_vertical
        
        U, S, V = svd(x[30000:])
        Phi, Lambda = dmd(x[30000:], U, S, V, x_prime[30000:], r)
        x_new_temp = X[30000:, seq_len_cache[instance]-end]
        x_pred_temp = Phi @ np.diag(Lambda) @ np.linalg.pinv(Phi) @ x_new_temp

    else:
        U, S, V = svd(x)
        Phi, Lambda = dmd(x, U, S, V, x_prime, r)
        x_new = X[:, seq_len_cache[instance]-end]
        
        # Discrete-time amplitude, if continuous-time is needed (refer to book).
        x_pred = Phi @ np.diag(Lambda) @ (np.linalg.pinv(Phi) @ x_new)
    
    x_new = X[:, seq_len_cache[instance]-end]
    x_ref = X[:, seq_len_cache[instance]-end+1]
    
    if seperate_modes:
        return np.hstack((x_pred_density, x_pred_horizonatal, x_pred_vertical, x_pred_temp)), x_ref, x_new
    else:
        return x_pred, x_ref, x_new

In [25]:
config.seperate_modes, config.remove_density = False, False

In [26]:
x_pred, x_ref, x_prev = dmd_prediction(file_cache, seq_len_cache, 1, 200, start=0, end=50, seperate_modes=config.seperate_modes, remove_density=config.remove_density)

In [27]:
x_prev_ref = np.load(file_cache['x_1_165'])

x_soln = np.load(file_cache['x_1_166'])
b_vec = np.load(file_cache['b_1_166'])
A_mat = sprse.load_npz(file_cache['A_1_166'])

In [28]:
if config.remove_density:
    print(f'Solutions Lineup: {(get_indpendent_vars(x_soln) == x_ref).sum() == 40000}, Previous Lineup: {(get_indpendent_vars(x_prev_ref) == x_prev).sum() == 40000}')
else:
    print(f'Solutions Lineup: {(x_soln == x_ref).sum() == 40000}, Previous Lineup: {(x_prev_ref == x_prev).sum() == 40000}')

Solutions Lineup: True, Previous Lineup: True


In [38]:
print(f'Density error: {np.linalg.norm(np.real(x_pred[:10000]) - x_ref[:10000])}')
print(f'Horizontal Velocity error: {np.linalg.norm(np.real(x_pred[10000:20000]) - x_ref[10000:20000])}')
print(f'Vertical Velocity error: {np.linalg.norm(np.real(x_pred[20000:30000]) - x_ref[20000:30000])}')
print(f'Temperature error: {np.linalg.norm(np.real(x_pred[30000:]) - x_ref[30000:])}')
print(f'Overall error: {np.linalg.norm(np.real(x_pred) - x_ref)}')

Density error: 0.009604269904375625
Horizontal Velocity error: 1.1743267358273675
Vertical Velocity error: 1.205499026419608
Temperature error: 0.28326528554141656
Overall error: 1.7066348904117832


In [30]:
if True:
    if config.remove_density:
        plot_timestep(x_ref, 't_ref')
        plot_timestep(x_pred, 't_pred')
        plot_timestep(x_prev, 't_prev')
    else:
        plot_timestep(get_indpendent_vars(x_ref), 't_ref')
        plot_timestep(get_indpendent_vars(x_pred), 't_pred')
        plot_timestep(get_indpendent_vars(x_prev), 't_prev')

/data/users/jupyter-dam724/.local/lib/python3.9/site-packages/numpy/ma/core.py:2820: ComplexWarning: Casting complex values to real discards the imaginary part
  _data = np.array(data, dtype=dtype, copy=copy,
/opt/tljh/user/lib/python3.9/site-packages/matplotlib/contour.py:1159: ComplexWarning: Casting complex values to real discards the imaginary part
  self.levels = np.asarray(levels_arg).astype(np.float64)


In [31]:
def get_solve_time(A_mat, b_vec, x0):
    iters = []

    def callback(norm):
        iters.append(norm)
    
    x_pred_soln, exit_code = sprse.linalg.gmres(A_mat, b_vec, x0, callback=callback)
    return len(iters)

In [32]:
if config.remove_density:
    print(f'Iterations: {get_solve_time(A_mat, b_vec, get_dependent_vars(x_pred))}')
    print(f'Prev Timestep: {get_solve_time(A_mat, b_vec, get_dependent_vars(x_prev))}')
else:
    print(f'Iterations: {get_solve_time(A_mat, b_vec, x_pred)}')
    print(f'Prev Timestep: {get_solve_time(A_mat, b_vec, x_prev)}')

/opt/tljh/user/lib/python3.9/site-packages/scipy/sparse/linalg/_isolve/utils.py:121: ComplexWarning: Casting complex values to real discards the imaginary part
  x = array(x0, dtype=xtype)


Iterations: 1256
Prev Timestep: 1590


# Sanity Check 02

Normalized data (with small noise) does not show issue with gmres solving time.

In [33]:
def normalize(x_soln, offset=0.0):
    x_temp = x_soln.reshape(4, 100, 100)
    
    mean = np.mean(x_soln.reshape(4, 100, 100), axis=(1, 2), keepdims=True)
    std = np.std(x_soln.reshape(4, 100, 100), axis=(1, 2), keepdims=True)
    
    nrom = ((x_temp - mean) / std) - (offset * np.random.rand(4, 100, 100))
    unorm = (nrom * std) + mean
    
    return unorm.reshape(40000)

In [34]:
np.linalg.norm(normalize(x_soln) - normalize(x_soln, offset=0.0003))

0.13812293360103767

In [35]:
print(f'(Norm -> Un-Norm) iterations: {get_solve_time(A_mat, b_vec, normalize(x_soln, offset=0.00005).astype(np.float32))}')

(Norm -> Un-Norm) iterations: 42
